In [ ]:
import pandas as pd
import numpy as np
import os

from datetime import datetime, timedelta

In [ ]:
filename = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Current_COOP_MEDALLION.csv'
df = pd.read_csv(filename, sep = ',')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')
df.columns = df.columns.str.replace('#', 'num')
df = df.assign(CreateDate=0)

ts_currentData = os.path.getctime(filename)
dt_currentData = datetime.fromtimestamp(ts_currentData) - timedelta(days=31) # USED FOR TESTING-POC *** THIS SHOULD BE CHANGED
tup_currentData = dt_currentData.timetuple()

print(tup_currentData)

In [ ]:
filename2 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/COOP_MEDALLION_Incentive.txt'
df_tierMap = pd.read_csv(filename2, sep = '\t')
df_tierMap.columns = df_tierMap.columns.str.replace(' ', '_')

ts_incentive = os.path.getctime(filename2)
dt_incentive = datetime.fromtimestamp(ts_incentive)
# print(dt_currentData)

In [ ]:
df['CreateDate'] = str(tup_currentData.tm_year) + '-' + str(tup_currentData.tm_mon) + '-' + str(tup_currentData.tm_mday)

In [ ]:
# df.columns.to_series(index=range(df.shape[1]))

In [ ]:
currentYear = datetime.now().year # use this for last transaction from G3X program
tup_now = datetime.now().timetuple() # used to create filename for CSV output

In [ ]:
calculatedData = pd.DataFrame([])
for ii in range(df.shape[0]):

    singleCusty = df.iloc[[ii]]
    singleCusty.reset_index(inplace=True)
        
    if  (singleCusty.Primary_Share.values >= 80) & ((singleCusty.Qualifying_Units_Purchased/singleCusty.YTD_Unit_Goal).values >= 0.5):

        if (tup_currentData.tm_mon == 1) | (tup_currentData.tm_mon == 2) | (tup_currentData.tm_mon == 3):
            if ((tup_currentData.tm_year % 400 == 0) | (tup_currentData.tm_year % 4 == 0)) & (tup_currentData.tm_year % 100 != 0):
                quarter = 1
                daysINqtr = 91
                runningDays = 91
                print('Leap year')
                singleCusty_select = singleCusty[
                ['Dealer_Name','Enrollment_Level','Primary_Share','Qualifying_Units_Purchased','YTD_Units_Purchased']]
            else:
                quarter = 1
                daysINqtr = 90
                runningDays = 90
                singleCusty_select = singleCusty[
                ['Dealer_Name','Enrollment_Level','Primary_Share','Qualifying_Units_Purchased','YTD_Units_Purchased']]
        elif (tup_currentData.tm_mon == 4) | (tup_currentData.tm_mon == 5) | (tup_currentData.tm_mon == 6):
            quarter = 2
            daysINqtr = 91
            runningDays = 90
            singleCusty_select = singleCusty[
            ['Dealer_Name','Enrollment_Level','Primary_Share','Qualifying_Units_Purchased','YTD_Units_Purchased']]
        elif (tup_currentData.tm_mon == 7) | (tup_currentData.tm_mon == 8) | (tup_currentData.tm_mon == 9):
            quarter = 3
            daysINqtr = 92
            runningDays = 90 + 91
            singleCusty_select = singleCusty[
            ['Dealer_Name','Enrollment_Level','Primary_Share','Qualifying_Units_Purchased','YTD_Units_Purchased']]
        else:
            quarter = 4
            daysINqtr = 92
            runningDays = 90 + 91 + 92
            singleCusty_select = singleCusty[
            ['Dealer_Name','Enrollment_Level','Primary_Share','Qualifying_Units_Purchased','YTD_Units_Purchased']]
        
        #########################################################################################################################
        if quarter in [2,3,4]:
            numDays = tup_currentData.tm_yday - runningDays # get number of days
        else:
            numDays = tup_currentData.tm_yday # get number of days
        #########################################################################################################################
        
        currentTires = singleCusty_select.YTD_Units_Purchased.values
        MinQty = df_tierMap[df_tierMap.Tier.values == singleCusty_select.Enrollment_Level.values].MinQty.values
        QualifyingTires = df_tierMap[df_tierMap.Tier.values == singleCusty_select.Enrollment_Level.values].Qualifying_Tires.values
        LevelBonus = df_tierMap[df_tierMap.Tier.values == singleCusty_select.Enrollment_Level.values].Level_Bonus.values
        
        if   currentTires >= MinQty:
            currentTier = 1
            currentRebate = (singleCusty_select.Qualifying_Units_Purchased * (QualifyingTires + LevelBonus)).values
        else:
            currentTier = 0
            currentRebate = 0
        
        estimatedTires = np.floor(singleCusty_select.iloc[:,4] / (numDays/daysINqtr)).values
        estimatedQualTires = np.floor(singleCusty_select.iloc[:,3] / (numDays/daysINqtr)).values
        if   estimatedTires >= MinQty:
            estTier = 1
            estRebate = estimatedQualTires * (QualifyingTires + LevelBonus)
        else:
            estTier = 0
            estRebate = 0

# ###########################################################################################################################
#         print(singleCusty_select)
# ###########################################################################################################################     
#         print('\nCurrent Tier:', currentTier)
#         print('\nCurrent Rebate:', currentRebate)
#         print('\nCurrent QTY:', currentTires)
        
#         print('\nEstimated Tier:', estTier)
#         print('\nEstimated Rebate:', estRebate)
#         print('\nEstimated QTY:', estimatedTires)
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################
#         print(tup_currentData, '\n' 'indexNumber:', ii)
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################

        tempData = pd.DataFrame(
            {'current_tier':currentTier, 'current_total_rebate':currentRebate, 'current_tires': currentTires,
             'estimated_tier':estTier, 'total_est_rebate':estRebate, 'current_est_tires':estimatedTires,
             'period': quarter, 'grain': 'Quarter'})
        calculatedData = calculatedData.append(tempData)
    else:
        tempData = pd.DataFrame(
            {'current_tier':0, 'current_total_rebate':0, 'current_tires': 0,
             'estimated_tier':0, 'total_est_rebate':0, 'current_est_tires':0,
             'period': [quarter], 'grain': 'Quarter'})
        calculatedData = calculatedData.append(tempData)
###########################################################################################################################        
calculatedData = calculatedData.reset_index(drop=True)
finalData = pd.concat([calculatedData,df], axis=1)

In [ ]:
finalData = finalData.replace([np.inf, -np.inf], np.nan).fillna(0)
# finalData.columns

In [ ]:
path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
writeFilename = path + 'CMP_COOP_MEDALLION.csv'
finalData.to_csv(writeFilename, index=False)
writeFilename_archive = path + 'csv_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
                        + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'CMP_COOP_MEDALLION_' \
                        + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
                        + '.csv'
finalData.to_csv(writeFilename_archive, index=False)

In [ ]:
# os.getcwd()